In [ ]:
import os
from pathlib import Path
import random

import pandas as pd
import numpy as np
import torch

import matplotlib.pyplot as plt


Data:
- [OPUS](https://opus.nlpl.eu/)
- [WMT](https://www.statmt.org/wmt20/)

In [ ]:
IMAGES_PATH = Path('imgs/')
DATA_PATH = Path('data/')

IMAGES_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
SEED = 42
BATCH_SIZE = 64

In [ ]:
def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

seed_all(SEED)

# Загрузка данных

In [ ]:
import requests
import gzip
import os
import xml.etree.ElementTree as ET
import pandas as pd

def download_and_unpack(url, output_path):
    """ Скачивание файла и его распаковка. """
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print("Download completed.")
        return True
    else:
        print("Failed to download file.")
        return False

def extract_gz(file_path, extract_to):
    """ Распаковка gz архива. """
    with gzip.open(file_path, 'rb') as f_in:
        with open(extract_to, 'wb') as f_out:
            f_out.write(f_in.read())
    print("Extraction completed.")

def read_tmx_to_dataframe(tmx_path):
    """ Чтение TMX файла и преобразование в DataFrame. """
    tree = ET.parse(tmx_path)
    root = tree.getroot()
    
    data = []
    for tu in root.findall('.//tu'):
        tuv = tu.findall('tuv')
        if len(tuv) >= 2:
            src_text = tuv[0].find('seg').text
            trg_text = tuv[1].find('seg').text
            data.append({'source': src_text, 'target': trg_text})
    
    df = pd.DataFrame(data)
    return df


In [ ]:
# Пример использования:
url = 'https://object.pouta.csc.fi/OPUS-wikimedia/v20230407/tmx/en-ru.tmx.gz'  # Замените на вашу ссылку

# Укажите URL архива и локальные пути для сохранения
download_path = os.path.join(DATA_PATH, 'en-ru.tmx.gz')
tmx_path = os.path.join(DATA_PATH, 'en-ru.tmx')

if download_and_unpack(url, download_path):
    extract_gz(download_path, tmx_path)
    dataframe = read_tmx_to_dataframe(tmx_path)
    print(dataframe.head())

    # Опционально: удаление временных файлов
    # os.remove(download_path)
    # os.remove(tmx_path)


In [ ]:
tmx_path = os.path.join(DATA_PATH, 'en-ru.tmx')
dataframe = read_tmx_to_dataframe(tmx_path)
dataframe = dataframe[:100]

In [ ]:
dataframe.head()

In [ ]:
dataframe.info()

## Препроцессинг

### Удаление пропусков

In [ ]:
# Очистка данных от пустых строк и None
dataframe = dataframe.dropna()  # Удаление строк, где есть хотя бы один None
dataframe = dataframe[dataframe['source'].str.strip().astype(bool) & dataframe['target'].str.strip().astype(bool)]  # Удаление пустых строк


# Датасет
## Токенизация и создание словарей

In [ ]:
import spacy
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm


class TextTokenizer:
    def __init__(self, lang, max_vocab_size=10000, min_freq=1, max_length=100):
        self.tokenizer = spacy.load(lang)
        self.max_vocab_size = max_vocab_size
        self.min_freq = min_freq
        self.max_length = max_length
        self.vocab = None
        
        self.specials = ['<unk>', '<pad>', '<bos>', '<eos>']

    def _tokenize(self, text):
        return ['<bos>'] +[token.text.lower() for token in self.tokenizer(text)] + ['<eos>']

    def build_vocab(self, data):
        token_stream = (self._tokenize(sentence) for sentence in tqdm(data, desc="Vocab build"))
        
        self.vocab = build_vocab_from_iterator(token_stream, max_tokens=self.max_vocab_size, specials=self.specials, special_first=True)
        self.vocab.set_default_index(self.vocab['<unk>'])

    def tokenize(self, text):
        tokenized_text = self._tokenize(text)[:self.max_length]
        return [self.vocab[token] for token in tokenized_text]

    def detokenize(self, numericalized_text):
        decoded_text = [self.vocab.get_itos()[index] \
            for index in numericalized_text \
                if self.vocab.get_itos()[index] not in self.specials]
        return ' '.join(decoded_text)


class TranslationDataset(Dataset):
    def __init__(self, 
                 dataframe: pd.DataFrame, 
                 src_col: str, 
                 trg_col: str, 
                 tokenizer_src: TextTokenizer, 
                 tokenizer_trg: TextTokenizer, 
                 padding_value=1):
        self.dataframe = dataframe
        self.src_col = src_col
        self.trg_col = trg_col
        self.tokenizer_src = tokenizer_src
        self.tokenizer_trg = tokenizer_trg
        self.padding_value = padding_value

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        src_sentence = self.dataframe.iloc[idx][self.src_col]
        trg_sentence = self.dataframe.iloc[idx][self.trg_col]
        tokenized_src = self.tokenizer_src.tokenize(src_sentence)
        tokenized_trg = self.tokenizer_trg.tokenize(trg_sentence)
        return {
            "src": torch.tensor(tokenized_src, dtype=torch.long),
            "trg": torch.tensor(tokenized_trg, dtype=torch.long)
        }

    def collate_fn(self, batch):
        src_batch = [item['src'] for item in batch]
        trg_batch = [item['trg'] for item in batch]
        src_padded = pad_sequence(src_batch, padding_value=self.padding_value, batch_first=True)
        trg_padded = pad_sequence(trg_batch, padding_value=self.padding_value, batch_first=True)
        return {"src": src_padded, "trg": trg_padded}


# Тест

In [ ]:
# # data = {'source': ["Hello world", "How are you?"], 'target': ["Привет мир", "Как у тебя дела?"]}
# # df = pd.DataFrame(data)

# # Создание токенизаторов и построение словарей
# tokenizer_src = TextTokenizer(lang='en_core_web_sm')
# tokenizer_trg = TextTokenizer(lang='ru_core_news_sm')
# tokenizer_src.build_vocab(dataframe['source'])
# tokenizer_trg.build_vocab(dataframe['target'])

# # Разделение данных на обучающие и тестовые наборы
# # train_df, val_df = train_test_split(dataframe, test_size=test_size, random_state=42)

# # Пример использования
# sample_text = "Hello world"
# numericalized = tokenizer_src.tokenize(sample_text)
# print("Numericalized:", numericalized)
# print("Detokenized:", tokenizer_src.detokenize(numericalized))


In [ ]:
# dataset = TranslationDataset(dataframe, 'source', 'target', tokenizer_src, tokenizer_trg)

# dataloader = DataLoader(dataset, batch_size=2, collate_fn=dataset.collate_fn)

# for bath in tqdm(dataloader, desc="Testing dataloader"):
#     pass

# Модель

## SelfAttention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    """
    Класс для реализации механизма Self-Attention.
    """
    def __init__(self, embed_size, heads, dropout_rate=0.1, scale_factor=None):
        super(SelfAttention, self).__init__()
        
        self.embed_size = embed_size  # Размерность эмбеддинга
        self.heads = heads  # Количество attention heads
        self.head_dim = embed_size // heads  # Размерность для каждой головы внимания
        self.scale_factor = scale_factor if scale_factor is not None \
            else (self.embed_size ** 0.5)

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size должен быть кратен количеству голов"

        # Инициализация весов для query, key и value
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        # Объединение результатов голов внимания
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, values, keys, queries, mask):
        # Получение размера пакета
        batch_size = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        # Разбиваем входные тензоры на головы
        values = values.reshape(batch_size, value_len, self.heads, self.head_dim)
        keys = keys.reshape(batch_size, key_len, self.heads, self.head_dim)
        queries = queries.reshape(batch_size, query_len, self.heads, self.head_dim)

        values = self.values(values)  # Применяем линейные преобразования к значениям
        keys = self.keys(keys)  # Применяем линейные преобразования к ключам
        queries = self.queries(queries)  # Применяем линейные преобразования к запросам

        # Multiplying queries and keys for attention scores (N, heads, query_len, key_len)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        
        # Apply mask to the attention scores
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float('-inf'))

        # Normalizing the attention scores using softmax
        attention = torch.softmax(energy / self.scale_factor, dim=-1)
        
        # Apply dropout to attention
        attention = self.dropout(attention)

        # Multiplying the attention scores with the values
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            batch_size, query_len, self.heads * self.head_dim
        )
                
        # Final linear layer
        out = self.fc_out(out)
        return out, attention


# TransformerBlock

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    """
    Класс для одного блока Transformer, включающий в себя Self-Attention и Feed Forward сеть.
    """
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)  # Создаем слой Self-Attention
        self.norm1 = nn.LayerNorm(embed_size)  # Слой нормализации для residual connection после attention
        self.norm2 = nn.LayerNorm(embed_size)  # Слой нормализации для residual connection после feed forward
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),  # Линейный слой
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(forward_expansion * embed_size, embed_size),  # Линейный слой
        )
        self.dropout = nn.Dropout(dropout)  # Слой dropout для регуляризации

    def forward(self, value, key, query, mask):
        """
        Прохождение входных данных через блок Transformer.

        Аргументы:
            value: Тензор значений размером (batch_size, seq_length, embed_size)
            key: Тензор ключей размером (batch_size, seq_length, embed_size)
            query: Тензор запросов размером (batch_size, seq_length, embed_size)
            mask: Маска внимания для исключения паддинга, размером (batch_size, 1, seq_length)

        Возвращает:
            out: Результат прохождения данных через блок Transformer
        """
        # Проходим через механизм Self-Attention
        attention_out, _ = self.attention(value, key, query, mask)
        # Применяем residual connection и нормализацию
        x = self.dropout(self.norm1(attention_out + query))
        # Проходим через feed forward сеть
        forward_out = self.feed_forward(x)
        # Применяем residual connection и нормализацию
        out = self.dropout(self.norm2(forward_out + x))
        return out


# Encoder

In [ ]:
class Encoder(nn.Module):
    """
    Класс для Encoder части Transformer, содержащий стек Transformer блоков.
    """
    def __init__(self, 
                 src_vocab_size, 
                 embed_size, 
                 num_layers, 
                 heads, 
                 device, 
                 forward_expansion, 
                 dropout, 
                 max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)  # Создаем эмбеддинги слов
        self.position_embedding = nn.Embedding(max_length, embed_size)  # Создаем позиционные эмбеддинги
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size, 
                    heads, 
                    dropout, 
                    forward_expansion
                )
                for _ in range(num_layers)
            ]
        )  # Создаем стек блоков Transformer

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        """
        Прохождение входных данных через стек блоков Transformer Encoder.

        Аргументы:
            x: Тензор входных данных размером (batch_size, src_seq_length)
            mask: Маска внимания для исключения паддинга, размером (batch_size, 1, src_seq_length)

        Возвращает:
            out: Результат прохождения данных через стек блоков Transformer Encoder
        """
        batch_size, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(batch_size, seq_length).to(self.device)  # Генерируем позиции
        
        # Получаем эмбеддинги слов и позиций и суммируем их
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

        # Проходим через стек блоков Transformer
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


# DecoderBlock

In [ ]:
class DecoderBlock(nn.Module):
    """
    Класс для блока Decoder, который включает в себя Self-Attention, Encoder-Decoder Attention и Feed Forward.
    """
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)  # Создаем слой Self-Attention
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.norm3 = nn.LayerNorm(embed_size)
        self.encoder_decoder_attention = SelfAttention(embed_size, heads)  # Создаем слой внимания между Decoder и Encoder
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),  # Линейный слой
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(forward_expansion * embed_size, embed_size),  # Линейный слой
        )
        self.dropout = nn.Dropout(dropout)  # Слой dropout для регуляризации
        self.device = device

    def forward(self, x, value, key, src_mask, trg_mask):
        """
        Прохождение входных данных через блок Decoder.

        Аргументы:
            x: Тензор входных данных размером (batch_size, trg_seq_length, embed_size)
            value: Тензор значений размером (batch_size, src_seq_length, embed_size)
            key: Тензор ключей размером (batch_size, src_seq_length, embed_size)
            src_mask: Маска внимания для исключения паддинга в Encoder, размером (batch_size, 1, src_seq_length)
            trg_mask: Маска внимания для исключения предсказаний из будущего, размером (batch_size, trg_seq_length, trg_seq_length)

        Возвращает:
            out: Результат прохождения данных через блок Decoder
        """
        # Проходим через механизм Self-Attention внутри Decoder
        attention_out, _ = self.attention(x, x, x, trg_mask)
        # Применяем residual connection и нормализацию
        query = self.dropout(self.norm1(attention_out + x))
        
        # Проходим через механизм внимания между Decoder и Encoder
        encoder_decoder_attention_out, _ = self.encoder_decoder_attention(value, key, query, src_mask)
        # Применяем residual connection и нормализацию
        query = self.dropout(self.norm2(encoder_decoder_attention_out + query))

        # Проходим через feed forward сеть
        forward_out = self.feed_forward(query)
        # Применяем residual connection и нормализацию
        out = self.dropout(self.norm3(forward_out + query))

        return out


# Decoder

In [ ]:
class Decoder(nn.Module):
    """
    Класс для Decoder части Transformer.
    """
    def __init__(self, 
        trg_vocab_size, 
        embed_size, 
        num_layers, 
        heads, 
        forward_expansion, 
        dropout, 
        device, 
        max_length):
        
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)  # Создаем эмбеддинги слов
        self.position_embedding = nn.Embedding(max_length, embed_size)  # Создаем позиционные эмбеддинги
        
        self.layers = nn.ModuleList([
            DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
            for _ in range(num_layers)
        ])  # Создаем стек блоков Decoder
        
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)  # Финальный линейный слой для предсказания слов
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        """
        Прохождение входных данных через Decoder модель Transformer.

        Аргументы:
            x: Тензор входных данных размером (batch_size, trg_seq_length)
            enc_out: Выходные данные Encoder размером (batch_size, src_seq_length, embed_size)
            src_mask: Маска внимания для исключения паддинга в Encoder, размером (batch_size, 1, src_seq_length)
            trg_mask: Маска внимания для исключения предсказаний из будущего, размером (batch_size, trg_seq_length, trg_seq_length)

        Возвращает:
            out: Результат прохождения данных через Decoder модель Transformer
        """
        batch_size, trg_seq_length = x.shape
        positions = torch.arange(0, trg_seq_length).expand(batch_size, trg_seq_length).to(self.device)  # Генерируем позиции

        # Получаем эмбеддинги слов и позиций и суммируем их
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        # Проходим через стек блоков Decoder
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)  # Применяем линейный слой для предсказания следующего слова

        return out


# Transformer

In [ ]:
class Transformer(nn.Module):
    """
    Класс, интегрирующий Encoder и Decoder в полную модель Transformer.
    """
    def __init__(self, 
                 src_vocab_size, 
                 trg_vocab_size, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 embed_size=256, 
                 num_layers=6, 
                 forward_expansion=4, 
                 heads=8, 
                 dropout=0, 
                 device="cuda", 
                 max_length=100):
        
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length
        )
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length
        )
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        """
        Создание маски для исключения влияния padding токенов в процессе внимания.

        Аргументы:
            src: Тензор исходных данных размером (batch_size, src_seq_length)

        Возвращает:
            src_mask: Маска для исключения padding токенов размером (batch_size, 1, 1, src_seq_length)
        """
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (batch_size, 1, 1, src_len) для работы с механизмом внимания
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        batch_size, trg_len = trg.size()
        no_peak_mask = torch.tril(torch.ones((1, trg_len, trg_len), device=self.device)).bool()
        pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_mask = no_peak_mask & pad_mask
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        """
        Полный проход через модель.

        Аргументы:
            src: Тензор исходных данных размером (batch_size, src_seq_length)
            trg: Тензор целевых данных размером (batch_size, trg_seq_length)

        Возвращает:
            output: Результат предсказания модели размером (batch_size, trg_seq_length, trg_vocab_size)
        """
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_out = self.encoder(src, src_mask)
        output = self.decoder(trg, enc_out, src_mask, trg_mask)
        return output


# Config

In [ ]:
class Config:
    src_vocab_size = 32000  # Предполагаем, что у нас есть 32K уникальных токенов для исходного языка
    trg_vocab_size = 32000  # и для целевого языка
    embed_size = 512        # Размер эмбеддингов
    num_layers = 6          # Количество слоев в энкодере и декодере
    forward_expansion = 4   # Коэффициент увеличения для полносвязного слоя
    heads = 8               # Количество голов в механизме многослойного внимания
    dropout = 0.1           # Вероятность dropout
    max_length = 100        # Максимальная длина последовательности
    min_freq = 2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')         # Устройство для тренировки: 'cuda' или 'cpu'
    learning_rate = 0.0005  # Скорость обучения
    batch_size = 64         # Размер батча для обучения
    clip = 1
    num_epochs = 10
    test_size = 0.1

# Валидация

In [ ]:
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score

class Evaluator:
    def __init__(self, 
                 tokenizer_src: TextTokenizer, 
                 tokenizer_trg: TextTokenizer,
                 weights={'bleu': 0.34, 'rouge': 0.33, 'meteor': 0.33}):
        """
        Инициализация Evaluator с токенизаторами для исходного и целевого языков.

        :param tokenizer_src: Токенизатор для исходного языка.
        :param tokenizer_trg: Токенизатор для целевого языка.
        :param weights: Веса.
        """
        self.tokenizer_src = tokenizer_src
        self.tokenizer_trg = tokenizer_trg
        self.weights = weights        
        self.rouge = Rouge()

    def evaluate(self, model, dataloader):
        """
        Оценивает модель на данных, загруженных через data_loader.

        :param model: Модель для оценки.
        :param data_loader: DataLoader, предоставляющий данные для оценки.
        :return: Словарь с результатами по метрикам.
        """
        model.eval()
        references = []
        hypotheses = []
        
        with torch.no_grad():
            for batch in dataloader:
                src  = batch['src']
                trg  = batch['trg']

                # Генерация вывода модели
                output = model(src, trg[:, :-1])  # Исключаем токен <eos>
                output = output.argmax(-1)
                
                # Детокенизация результата
                for true, pred in zip(trg, output):
                    true_sentence = self.tokenizer_trg.detokenize(true.numpy())
                    pred_sentence = self.tokenizer_trg.detokenize(pred.numpy())
                    references.append([true_sentence])
                    hypotheses.append(pred_sentence)

        # Вычисление метрик BLEU, ROUGE, METEOR
        bleu_score = corpus_bleu(references, hypotheses)
        # rouge_scores = self.rouge.get_scores(hypotheses, [' '.join(ref[0]) for ref in references], avg=True)
        # rouge_score = rouge_scores['rouge-l']['f']
        rouge_score = 0
        # list_meteor_score = [meteor_score([ref[0]], hyp) for ref, hyp in zip(references, hypotheses)]
        # avg_meteor_score = np.mean(list_meteor_score)
        avg_meteor_score = 0

        # Словарь с результатами
        results = {
            'overall': self.weights['bleu'] * bleu_score +
                       self.weights['rouge'] * rouge_score +
                       self.weights['meteor'] * avg_meteor_score,
            'bleu': bleu_score,
            'rouge': rouge_score,
            'meteor': avg_meteor_score
        }
        return results
    
    def evaluate_on_indices(self, model, dataloader):
        """
        Оценивает модель на данных, используя индексы токенов для вычисления BLEU.

        :param model: Модель для оценки.
        :param data_loader: DataLoader, предоставляющий данные для оценки.
        :return: Словарь с результатами метрики BLEU.
        """
        model.eval()
        references = []
        hypotheses = []

        with torch.no_grad():
            for batch in dataloader:
                src  = batch['src']
                trg  = batch['trg']
                
                # Генерация вывода модели
                output = model(src, trg[:, :-1])
                output = output.argmax(-1)
                
                # Сохранение индексов без детокенизации
                for true, pred in zip(trg, output):
                    # Убираем индексы для специальных токенов
                    true_indices = [token for token in true.numpy() \
                        if token not in [self.tokenizer_trg.vocab[spec] \
                            for spec in self.tokenizer_trg.specials]]
                    pred_indices = [token for token in pred.numpy() \
                        if token not in [self.tokenizer_trg.vocab[spec] \
                            for spec in self.tokenizer_trg.specials]]
                    
                    references.append([true_indices])
                    hypotheses.append(pred_indices)

        # Вычисление метрик BLEU, ROUGE, METEOR
        bleu_score = corpus_bleu(references, hypotheses)
        rouge_score = self.rouge.get_scores(hypotheses, references, avg=True)['f']
        # list_meteor_score = [meteor_score([ref], hyp) for ref, hyp in zip(references, hypotheses)]
        # avg_meteor_score = np.mean(list_meteor_score)
        avg_meteor_score = 0

        # Словарь с результатами
        results = {
            'overall': self.weights['bleu'] * bleu_score +
                       self.weights['rouge'] * rouge_score +
                       self.weights['meteor'] * avg_meteor_score,
            'bleu': bleu_score,
            'rouge': rouge_score,
            'meteor': avg_meteor_score
        }
        return results


# Обучение

In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


class Trainer:
    def __init__(self, 
                 model:Transformer, 
                 evaluator: Evaluator, 
                 optimizer:nn.CrossEntropyLoss,
                 criterion:optim.Adam,
                 clip=1,
                 device='cuda',
                 printing_period=None,
                 imgs_path=Path(''),
                 ):
        """
        Инициализация Trainer для модели Transformer.
        """
        self.model = model.to(device)
        self.device = device
        self.evaluator = evaluator
        self.clip = clip
        
        self.printing_period = printing_period
        self.imgs_path = imgs_path
        self.train_losses = []
        self.val_losses = []
        self.metrics = []

        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, train_dataloader):
        """
        Обучение модели
        """
        self.model.train()
        train_loss = 0.0
        for batch in tqdm(train_dataloader, desc="Training"):
            src: Tensor = batch['src'].to(self.device)
            trg: Tensor = batch['trg'].to(self.device)
            
            self.optimizer.zero_grad()
            output: Tensor = self.model(src, trg[:, :-1])
            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            trg = trg[:, 1:].contiguous().view(-1)

            loss = self.criterion(output, trg)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)
            self.optimizer.step()
            
            # Аккумулируем потери
            train_loss += loss.item()

        # Вычисляем среднюю потерю за эпоху
        avg_loss = train_loss / len(train_dataloader)
        self.train_losses.append(avg_loss)
        return avg_loss

    def eval(self, val_dataloader):
        """
        Валидация модели для оценки производительности на валидационном наборе данных.
        """
        self.model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validating"):
                src: Tensor= batch['src'].to(self.device)
                trg: Tensor = batch['trg'].to(self.device)     
                           
                output: Tensor = self.model(src, trg[:, :-1])
                output_dim = output.shape[-1]

                output = output.contiguous().view(-1, output_dim)
                trg = trg[:, 1:].contiguous().view(-1)

                loss = self.criterion(output, trg)
                validation_loss += loss.item()
                
        avg_loss = validation_loss / len(val_dataloader)
        self.val_losses.append(avg_loss)
        
        metric_scores = self.evaluator.evaluate(self.model, val_dataloader)
        self.metrics.append(metric_scores)
        
        return avg_loss, metric_scores
        
    def fit(self, num_epochs, train_dataloader, val_dataloader):
        for epoch in range(num_epochs):
            epoch_loss = self.train(train_dataloader)
            val_epoch_loss, metric_scores = self.eval(val_dataloader)   
                 
            if self.printing_period and (epoch + 1) % self.printing_period == 0:
                print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Overall: {metric_scores["overall"]:.4f}')
                
        return val_epoch_loss, metric_scores

    def save_model(self, path):
        """
        Сохранение обученной модели.

        Параметры:
            path: путь для сохранения модели.
        """
        torch.save(self.model.state_dict(), path)

    def load_model(self, path):
        """
        Загрузка модели из файла.

        Параметры:
            path: путь к файлу с сохраненной моделью.
        """
        self.model.load_state_dict(torch.load(path))
        self.model.to(self.device)
        
    def _save_fig(self, fig_id, tight_layout=True, fig_extension="png", resolution=300):
        path = self.imgs_path / f"{fig_id}.{fig_extension}"
        if tight_layout:
            plt.tight_layout()
        plt.savefig(path, format=fig_extension, dpi=resolution) 

    def plot_metrics(self):
        epochs_range = range(1, len(self.val_losses) + 1)
        fig = plt.figure(figsize=(15, 6))  # Устанавливаем размер фигуры
        
        ax11 = plt.subplot(2, 2, 1)
        ax11.plot(epochs_range, self.train_losses, label='Train Loss', color='tab:red')
        ax11.plot(epochs_range, self.val_losses, label='Validation Loss', color='tab:blue')
        ax11.set_title('Losses over Epochs')
        ax11.set_xlabel('Epochs')
        ax11.set_ylabel('Loss')
        # ax11.tick_params(axis='y', labelcolor='tab:red')
        ax11.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax11.legend(loc='upper right')

        ax12 = plt.subplot(2, 2, 2)
        ax12.plot(epochs_range, [m['overall'] for m in self.metrics], label='Overall Score', color='tab:green')
        ax12.set_title('Overall Evaluation Score')
        ax12.set_xlabel('Epochs')
        ax12.set_ylabel('Score')
        ax12.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax12.legend(loc='upper right')

        # Отдельные графики для каждой метрики
        ax21 = plt.subplot(2, 2, 3)
        ax21.plot(epochs_range, [m['bleu'] for m in self.metrics], label='BLEU Score', color='tab:red')
        ax21.plot(epochs_range, [m['rouge'] for m in self.metrics], label='ROUGE Score', color='tab:pink')
        ax21.plot(epochs_range, [m['meteor'] for m in self.metrics], label='METEOR Score', color='tab:brown')
        ax21.set_title('Individual Metrics')
        ax21.set_xlabel('Epochs')
        ax21.set_ylabel('Score')
        ax21.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax21.legend(loc='upper right')      

        fig.tight_layout()  # Убедимся, что макет не нарушен
        self._save_fig("train_metrics")  # extra code
        plt.show()

In [ ]:
import pickle

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

class TrainingManager:
    def __init__(self, 
                 dataframe: pd.DataFrame,
                 features: str, 
                 targets: str,
                 config: Config,
                 data_path=DATA_PATH,
                 imgs_path=IMAGES_PATH):
        """
        Инициализация менеджера обучения.        
        :param features: Признаки для обучения модели.
        :param targets: Целевые значения.
        :param model_params: Параметры модели, включая размеры слоёв и функцию активации.
        :param train_params: Параметры обучения, включая размер батча, количество эпох и скорость обучения.
        """
        self.uniq_name = '_'.join(features + targets)
        self.data_path = data_path  
        self.imgs_path = imgs_path    
        
        self.features = features
        self.targets = targets
        self.dataframe = dataframe
        
        self.config = config
        
        self.tokenizer_src = TextTokenizer(
            lang='en_core_web_sm',
            max_vocab_size=config.src_vocab_size,
            min_freq=config.min_freq,
            max_length=config.max_length    
        )
        self.tokenizer_trg = TextTokenizer(
            lang='ru_core_news_sm',
            max_vocab_size=config.trg_vocab_size,
            min_freq=config.min_freq,
            max_length=config.max_length
        )
        self.tokenizer_src.build_vocab(dataframe['source'])
        self.tokenizer_trg.build_vocab(dataframe['target'])

        train_df, val_df = train_test_split(dataframe, test_size=config.test_size, random_state=42)

        train_dataset = TranslationDataset(
            dataframe=train_df,
            src_col='source',
            trg_col='target',
            tokenizer_src=self.tokenizer_src,
            tokenizer_trg=self.tokenizer_trg,
            padding_value=self.tokenizer_src.vocab['<pad>'],
        )

        val_dataset = TranslationDataset(
            dataframe=val_df,
            src_col='source',
            trg_col='target',
            tokenizer_src=self.tokenizer_src,
            tokenizer_trg=self.tokenizer_trg,
            padding_value=self.tokenizer_src.vocab['<pad>'],
        )

        self.train_dataloader = DataLoader(
            train_dataset, 
            batch_size=config.batch_size, 
            collate_fn=train_dataset.collate_fn, 
            shuffle=True
        )
        self.val_dataloader = DataLoader(
            val_dataset, 
            batch_size=config.batch_size, 
            collate_fn=val_dataset.collate_fn, 
            shuffle=False
        )

        # Шаг 1: Определение модели
        model = Transformer(
            src_vocab_size=len(self.tokenizer_src.vocab),
            trg_vocab_size=len(self.tokenizer_trg.vocab),
            src_pad_idx=self.tokenizer_src.vocab['<pad>'],
            trg_pad_idx=self.tokenizer_trg.vocab['<pad>'],
            embed_size=config.embed_size,
            num_layers=config.num_layers,
            forward_expansion=config.forward_expansion,
            heads=config.heads,
            dropout=config.dropout,
            device=config.device
        )

        # Шаг 2: Определение функции потерь
        criterion = nn.CrossEntropyLoss(ignore_index=self.tokenizer_src.vocab['<pad>'])

        # Шаг 3: Определение оптимизатора
        optimizer = Adam(model.parameters(), lr=0.0001)
        
        evaluator = Evaluator(
            self.tokenizer_src,
            self.tokenizer_trg,    
        )
        
        self.trainer = Trainer(
            model,
            evaluator,
            optimizer,
            criterion,
            clip=config.clip,
            device=config.device,
            printing_period=1,
            imgs_path=self.imgs_path    
        )  
    
    def fit(self):
        """
        Запуск процесса обучения.
        """
        
        self.trainer.fit(self.config.num_epochs, self.train_dataloader, self.val_dataloader)
        self.trainer.plot_metrics()

    # def predict(self, input, transform=True):
    #     """
    #     Выполнение предсказаний с помощью обученной модели.
    #     """
    #     features = input
    #     if transform:
    #         features = self.scaler_features.transform(input)
    #     else:
    #         features = input
    #     scaled_predictions = self.trainer.predict(features)
    #     return self.scaler_targets.inverse_transform(scaled_predictions)

    # def save(self):
    #     self._save_model()
    #     self._save_standard_scalar()
 
    # def load(self):
    #     self._load_model()
    #     self._load_standard_scalar()
    
    # def _save_model(self):
    #     """
    #     Сохранение обученной модели.        
    #     """
    #     self.trainer.save_model(self.data_path+self.uniq_name+'_model_weight.pth')
        
    # def _load_model(self):
    #     """
    #     Загрузка обученной модели.        
    #     """
    #     self.trainer = Trainer.load_model(self.data_path+self.uniq_name+'_model_weight.pth')


In [ ]:
config = Config

config.src_vocab_size = 20000  # Предполагаем, что у нас есть 32K уникальных токенов для исходного языка
config.trg_vocab_size = 20000  # и для целевого языка
config.embed_size = 64         # Размер эмбеддингов
config.num_layers = 6          # Количество слоев в энкодере и декодере
config.forward_expansion = 4   # Коэффициент увеличения для полносвязного слоя
config.heads = 8               # Количество голов в механизме многослойного внимания
config.dropout = 0.1           # Вероятность dropout
config.min_freq = 2
config.max_length = 100        # Максимальная длина последовательности
config.device = "cpu"          # Устройство для тренировки: 'cuda' или 'cpu'
config.learning_rate = 0.005  # Скорость обучения
config.batch_size = 128          # Размер батча для обучения
config.clip = 1                # 
config.num_epochs = 25
config.test_size = 0.1

In [ ]:

# import torch
# import torch.nn as nn
# from torch.optim import Adam
# from tqdm.auto import tqdm
# from sklearn.model_selection import train_test_split


# # Определение гиперпараметров
# max_vocab_size = 10000
# min_freq = 2
# max_length = 80
# embed_size = 64
# num_layers = 3
# forward_expansion = 2
# heads = 4
# dropout = 0.1
# learning_rate = 0.001
# batch_size = 256
# num_epochs = 10
# clip = 1
# test_size = 0.1
# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Создание токенизаторов и построение словарей
# tokenizer_src = TextTokenizer(
#     lang='en_core_web_sm',
#     max_vocab_size=max_vocab_size,
#     min_freq=min_freq,
#     max_length=max_length    
# )
# tokenizer_trg = TextTokenizer(
#     lang='ru_core_news_sm',
#     max_vocab_size=max_vocab_size,
#     min_freq=min_freq,
#     max_length=max_length
# )
# tokenizer_src.build_vocab(dataframe['source'])
# tokenizer_trg.build_vocab(dataframe['target'])

# train_df, val_df = train_test_split(dataframe, test_size=test_size, random_state=42)

# train_dataset = TranslationDataset(
#     dataframe=train_df,
#     src_col='source',
#     trg_col='target',
#     tokenizer_src=tokenizer_src,
#     tokenizer_trg=tokenizer_trg,
#     padding_value=tokenizer_src.vocab['<pad>'],
# )

# val_dataset = TranslationDataset(
#     dataframe=val_df,
#     src_col='source',
#     trg_col='target',
#     tokenizer_src=tokenizer_src,
#     tokenizer_trg=tokenizer_trg,
#     padding_value=tokenizer_src.vocab['<pad>'],
# )

# train_dataloader = DataLoader(
#     train_dataset, 
#     batch_size=batch_size, 
#     collate_fn=train_dataset.collate_fn, 
#     shuffle=True
# )
# val_dataloader = DataLoader(
#     val_dataset, 
#     batch_size=batch_size, 
#     collate_fn=val_dataset.collate_fn, 
#     shuffle=False
# )

# # Шаг 1: Определение модели
# model = Transformer(
#     src_vocab_size=len(tokenizer_src.vocab),
#     trg_vocab_size=len(tokenizer_trg.vocab),
#     src_pad_idx=tokenizer_src.vocab['<pad>'],
#     trg_pad_idx=tokenizer_trg.vocab['<pad>'],
#     embed_size=embed_size,
#     num_layers=num_layers,
#     forward_expansion=forward_expansion,
#     heads=heads,
#     dropout=dropout,
#     device=device
# )

# # Шаг 2: Определение функции потерь
# criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.vocab['<pad>'])

# # Шаг 3: Определение оптимизатора
# optimizer = Adam(model.parameters(), lr=0.0001)

# evaluator = Evaluator(
#     tokenizer_src,
#     tokenizer_trg,    
# )


In [ ]:

# # Шаг 4: Обучение модели
# def train(model: Transformer, iterator, optimizer: Adam, criterion: nn.CrossEntropyLoss, clip):
#     model.train()
#     epoch_loss = 0

#     for batch in tqdm(iterator, desc="Training"):
#         src = batch["src"]
#         trg = batch["trg"]

#         optimizer.zero_grad()
#         output = model(src, trg[:, :-1])
#         output_dim = output.shape[-1]

#         output = output.contiguous().view(-1, output_dim)
#         trg = trg[:, 1:].contiguous().view(-1)

#         loss = criterion(output, trg)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
#         optimizer.step()

#         epoch_loss += loss.item()

#     return epoch_loss / len(iterator)

# def evaluate(model, iterator, criterion):
#     model.eval()
#     epoch_loss = 0

#     with torch.no_grad():
#         for batch in tqdm(iterator, desc="Evaluation"):
#             src = batch["src"]
#             trg = batch["trg"]

#             output = model(src, trg[:, :-1])
#             output_dim = output.shape[-1]

#             output = output.contiguous().view(-1, output_dim)
#             trg = trg[:, 1:].contiguous().view(-1)

#             loss = criterion(output, trg)
#             epoch_loss += loss.item()

#     return epoch_loss / len(iterator)

# CLIP = 1

# best_valid_loss = float('inf')

# for epoch in range(num_epochs):
#     train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
#     valid_loss = evaluate(model, val_dataloader, criterion)

#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'transformer_model.pt')

#     print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')


In [ ]:
# trainer = Trainer(
#     model,
#     evaluator,
#     optimizer,
#     criterion,
#     clip=clip,
#     device=device,
#     printing_period=1,
#     imgs_path=IMAGES_PATH    
# )
# trainer.fit(
#     num_epochs=num_epochs,
#     train_dataloader=train_dataloader,
#     val_dataloader=val_dataloader,
# )


In [ ]:
# trainer.plot_metrics()

In [ ]:
trainer_manager = TrainingManager(
    dataframe,
    'source',
    'target',
    config
)


In [ ]:

trainer_manager.fit()

# Предикт

# Выполнение

# Решение